In [1]:
import requests as r
import pandas as pd
import os

import sys
import csv
sys.path.append("../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

'../helper_functions'

In [2]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# start_date = '2024-03-01'
trailing_days = 7

chain_names = ['op', 'base']

In [3]:
query = '''
SELECT *
FROM @chain_name@_transactions
WHERE gas_price > 0
        AND block_timestamp >= DATE_TRUNC('day',NOW()) - interval '@trailing_days@ days'
        AND block_timestamp < DATE_TRUNC('day',NOW())
        AND receipt_status = 1

SETTINGS max_execution_time = 5000
'''


In [4]:
for chain_name in chain_names:
        print(chain_name)
        q_run = query.replace('@trailing_days@', str(trailing_days))
        q_run = q_run.replace('@chain_name@', chain_name)

        # print(q_run)
        
        result = client.query_df(q_run)
        # start_date_file = start_date.replace('-', '_')
        result_df.to_csv('csv_outputs/' + chain_name + '_' + trailing_days)

op


AttributeError: 'QueryResult' object has no attribute 'column_descriptions'

In [9]:
# print(result.result_rows)

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))